Задание 4. Точные методы решения СЛАУ (LU-разложение).

In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import hilbert
from numpy import linalg as LA

In [2]:
def LU_decomposition(matrix: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    size = matrix.shape[0]
    L = np.identity(size)
    U = np.array(matrix, dtype=float)

    for column in range(size - 1):
        for row in range(column + 1, size):
            L[row, column] = U[row, column] / U[column, column]
            U[row] -= L[row, column] * U[column]
    return L, U

In [3]:
def forward_substitution(L: np.ndarray, b: np.array) -> np.array:
    y = []
    for i in range(L.shape[0]):
        y.append(b[i] - sum(L[i, :i] * y))
    return np.array(y)


def backward_substitution(U: np.ndarray, y: np.array) -> np.array:
    x = []
    for i in range(U.shape[0] - 1, -1, -1):
        x.insert(0, (y[i] - sum(U[i, i + 1:] * x)) / U[i, i])
    return np.array(x)

In [4]:
def spectral_condition_number(matrix: np.ndarray) -> float:
    return LA.norm(matrix) * LA.norm(LA.inv(matrix))


def volume_condition_number(matrix: np.ndarray) -> float:
    volume = 1
    for row in matrix:
        volume *= LA.norm(row)
    return volume / abs(LA.det(matrix))


def angle_condition_number(matrix: np.ndarray) -> float:
    size = matrix.shape[0]
    inverse_matrix_t = LA.inv(matrix).T
    return max(LA.norm(matrix[n]) * LA.norm(inverse_matrix_t[n]) for n in range(0, size))


condition_numbers = {
    'cond_s': spectral_condition_number,
    'cond_v': volume_condition_number,
    'cond_a': angle_condition_number
}

In [5]:
columns = ['A', 'L', 'U']

index = condition_numbers.keys()

def display_result(matrix: np.ndarray):
    L, U = LU_decomposition(matrix)
    matrices = [matrix, L, U]
    data = [[condition_numbers[i](m) for m in matrices] for i in index]
    display(pd.DataFrame(data, index=index, columns=columns))

    exact_x = np.ones(matrix.shape[0])
    b = matrix @ exact_x
    
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)

    print(f"Погрешность решения: {LA.norm(exact_x - x)}")

### Матрица Гильберта

In [6]:
display_result(hilbert(8))

,A,L,U
cond_s,1.549362e+10,132.117616,1.556130e+09
cond_v,7.564448e+29,2605.736682,1.137521e+04
cond_a,2.025215e+09,25.018643,2.326438e+01


Погрешность решения: 2.2003668835673837e-06


### Диагональная матрица

In [7]:
display_result(np.diag(range(-5, 0)))

,A,L,U
cond_s,8.972102,5.0,8.972102
cond_v,1.000000,1.0,1.000000
cond_a,1.000000,1.0,1.000000


Погрешность решения: 0.0


### Трехдиагональная матрица с диагональным преобладанием 

In [8]:
size = 10
matrix = 2 * np.identity(size) + -1 * np.eye(size, k=-1) + -1 * np.eye(size, k=1)
display_result(matrix)

,A,L,U
cond_s,98.122373,19.333982,21.190056
cond_v,589.090909,8.820485,8.820485
cond_a,12.898203,2.292499,2.523592


Погрешность решения: 1.1102230246251565e-16
